In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("official/mnist/dataset.py", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting official/mnist/dataset.py\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting official/mnist/dataset.py\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting official/mnist/dataset.py\t10k-images-idx3-ubyte.gz
Extracting official/mnist/dataset.py\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
## Initialise weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [5]:
## Initialise bias
def init_bias(shape):
    init_bias_values = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_values)

In [6]:
## Create a 2D convolution function
def convTwoDim(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [7]:
def max_pool(x):  ## 2x2 pool function
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [8]:
def convolution_layer(x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.sigmoid(convTwoDim(x, W))

In [23]:
def fully_connected_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

## Placeholders

In [11]:
x = tf.placeholder(tf.float32, shape=[None,784])

In [12]:
y_true = tf.placeholder(tf.float32, shape=[None,10])

## Layers

In [14]:
x_image = tf.reshape(x, [-1,28,28,1])

In [16]:
##Layer 1
conv_lay_1 = convolution_layer(x_image, shape=[5,5,1,32])
conv_lay_1_pool = max_pool(conv_lay_1)

In [18]:
##Layer 2
conv_lay_2 = convolution_layer(conv_lay_1_pool, shape=[5,5,32,64])
conv_lay_2_pool = max_pool(conv_lay_2)

In [56]:
conv_2_flat = tf.reshape(conv_lay_2_pool, [-1,7*7*64])
full_layer_1 = tf.nn.sigmoid(fully_connected_layer(conv_2_flat, 1024))

In [57]:
##Drop Out
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_1, keep_prob=hold_prob)

In [58]:
y_pred = fully_connected_layer(full_one_dropout, 10)

In [59]:
## LOSS FUNCTION
cross_entropy =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))

In [60]:
## Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [61]:
init = tf.global_variables_initializer()

In [63]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            print("On step ", i)
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
            print(sess.run(accuracy, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}), "\n")

On step  0
Accuracy: 
0.0982 

On step  100
Accuracy: 
0.1009 

On step  200
Accuracy: 
0.7399 

On step  300
Accuracy: 
0.8871 

On step  400
Accuracy: 
0.8922 

On step  500
Accuracy: 
0.9191 

On step  600
Accuracy: 
0.9348 

On step  700
Accuracy: 
0.94 

On step  800
Accuracy: 
0.9476 

On step  900
Accuracy: 
0.9495 

On step  1000
Accuracy: 
0.9543 

On step  1100
Accuracy: 
0.9573 

On step  1200
Accuracy: 
0.9634 

On step  1300
Accuracy: 
0.9601 

On step  1400
Accuracy: 
0.9657 

On step  1500
Accuracy: 
0.9673 

On step  1600
Accuracy: 
0.9702 

On step  1700
Accuracy: 
0.9734 

On step  1800
Accuracy: 
0.9732 

On step  1900
Accuracy: 
0.976 

On step  2000
Accuracy: 
0.9716 

On step  2100
Accuracy: 
0.9763 

On step  2200
Accuracy: 
0.9725 

On step  2300
Accuracy: 
0.9783 

On step  2400
Accuracy: 
0.978 

On step  2500
Accuracy: 
0.9764 

On step  2600
Accuracy: 
0.9762 

On step  2700
Accuracy: 
0.9787 

On step  2800
Accuracy: 
0.9803 

On step  2900
Accuracy: 
0.982